## 勾配ブースティングを使って文書分類モデルを実装

In [1]:
import os
from glob import glob
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# カテゴリ（ディレクトリ名）をリスト化
categories = [name for name in os.listdir('text') if os.path.isdir("text/" +name)]
print(categories)

# カテゴリをID化
category2id = {}
for i, cat in enumerate(categories):
    category2id[cat] = i

# DataFrame作成
datasets = pd.DataFrame(columns=["document", "category"])

for category in tqdm(categories):
    path = "text/" + category + "/*.txt"
    files = glob(path)
    for text_name in files:
        with open(text_name, 'r', encoding='utf-8') as f:
            document = f.read()
            row = pd.Series([document, category], index=datasets.columns)
            datasets = datasets.append(row, ignore_index=True)
print("doc num", len(datasets))


['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



doc num 7376


#### 形態素解析の定義

In [2]:
import MeCab
import re

tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
    sentence = tagger.parse(sentence)
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=＝)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    wakati = sentence.split(" ")
    wakati = list(filter(("").__ne__, wakati))
    return wakati

#### Word2Vecで単語分散表現作成

In [3]:
from gensim.models import Word2Vec
import numpy as np
import logging

# word2vec parameters
num_features    = 200
min_word_count  = 5
num_workers     = 40
context         = 10
downsampling    = 1e-3
model_name = "livedoor_corpus_feature200.model"

# コーパス読み込み
corpus = []
for doc in tqdm(datasets["document"]):
    corpus.append(make_wakati(doc))


# word2vecモデルの作成＆モデルの保存
print("cleating word2vec model ...")
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = Word2Vec(corpus, workers=num_workers, hs = 0, sg = 1, negative = 10, iter = 25, 
                 size=num_features, min_count = min_word_count,  window = context, sample = downsampling, seed=1)
model.save(model_name)
print("Done.")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


2021-03-06 13:31:11,821 : INFO : collecting all words and their counts
2021-03-06 13:31:11,839 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types



cleating word2vec model ...


2021-03-06 13:31:13,783 : INFO : collected 59937 word types from a corpus of 3848224 raw words and 7376 sentences
2021-03-06 13:31:13,791 : INFO : Loading a fresh vocabulary
2021-03-06 13:31:14,962 : INFO : effective_min_count=5 retains 24994 unique words (41% of original 59937, drops 34943)
2021-03-06 13:31:14,971 : INFO : effective_min_count=5 leaves 3786969 word corpus (98% of original 3848224, drops 61255)
2021-03-06 13:31:15,497 : INFO : deleting the raw counts dictionary of 59937 items
2021-03-06 13:31:15,506 : INFO : sample=0.001 downsamples 34 most-common words
2021-03-06 13:31:15,516 : INFO : downsampling leaves estimated 2737656 word corpus (72.3% of prior 3786969)
2021-03-06 13:31:15,711 : INFO : estimated required memory for 24994 words and 200 dimensions: 52487400 bytes
2021-03-06 13:31:15,718 : INFO : resetting layer weights
2021-03-06 13:31:33,112 : INFO : training model with 40 workers on 24994 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=10 window

2021-03-06 13:34:17,846 : INFO : EPOCH 1 - PROGRESS: at 73.41% examples, 13498 words/s, in_qsize 75, out_qsize 0
2021-03-06 13:34:19,920 : INFO : EPOCH 1 - PROGRESS: at 73.59% examples, 13369 words/s, in_qsize 74, out_qsize 0
2021-03-06 13:34:21,448 : INFO : EPOCH 1 - PROGRESS: at 73.98% examples, 13324 words/s, in_qsize 71, out_qsize 1
2021-03-06 13:34:23,436 : INFO : EPOCH 1 - PROGRESS: at 74.81% examples, 13326 words/s, in_qsize 68, out_qsize 0
2021-03-06 13:34:24,504 : INFO : EPOCH 1 - PROGRESS: at 75.22% examples, 13325 words/s, in_qsize 66, out_qsize 0
2021-03-06 13:34:26,091 : INFO : EPOCH 1 - PROGRESS: at 75.89% examples, 13325 words/s, in_qsize 63, out_qsize 0
2021-03-06 13:34:27,537 : INFO : EPOCH 1 - PROGRESS: at 76.91% examples, 13372 words/s, in_qsize 56, out_qsize 3
2021-03-06 13:34:28,596 : INFO : EPOCH 1 - PROGRESS: at 77.18% examples, 13333 words/s, in_qsize 49, out_qsize 9
2021-03-06 13:34:29,716 : INFO : EPOCH 1 - PROGRESS: at 81.10% examples, 13644 words/s, in_qsize

...

2021-03-06 14:48:53,691 : INFO : EPOCH 24 - PROGRESS: at 11.28% examples, 11330 words/s, in_qsize 80, out_qsize 15
2021-03-06 14:49:01,462 : INFO : EPOCH 24 - PROGRESS: at 15.13% examples, 11803 words/s, in_qsize 79, out_qsize 0
2021-03-06 14:49:02,908 : INFO : EPOCH 24 - PROGRESS: at 16.40% examples, 12162 words/s, in_qsize 80, out_qsize 2
2021-03-06 14:49:04,319 : INFO : EPOCH 24 - PROGRESS: at 18.14% examples, 12795 words/s, in_qsize 79, out_qsize 2
2021-03-06 14:49:05,538 : INFO : EPOCH 24 - PROGRESS: at 19.58% examples, 13311 words/s, in_qsize 79, out_qsize 0
2021-03-06 14:49:07,179 : INFO : EPOCH 24 - PROGRESS: at 19.94% examples, 13161 words/s, in_qsize 80, out_qsize 1
2021-03-06 14:49:08,550 : INFO : EPOCH 24 - PROGRESS: at 20.93% examples, 13334 words/s, in_qsize 78, out_qsize 1
2021-03-06 14:49:09,895 : INFO : EPOCH 24 - PROGRESS: at 22.30% examples, 13767 words/s, in_qsize 76, out_qsize 3
2021-03-06 14:49:20,089 : INFO : EPOCH 24 - PROGRESS: at 25.04% examples, 12683 words/s

2021-03-06 14:51:19,707 : INFO : EPOCH 24 - PROGRESS: at 88.56% examples, 13762 words/s, in_qsize 29, out_qsize 1
2021-03-06 14:51:19,711 : INFO : worker thread finished; awaiting finish of 29 more threads
2021-03-06 14:51:19,783 : INFO : worker thread finished; awaiting finish of 28 more threads
2021-03-06 14:51:19,970 : INFO : worker thread finished; awaiting finish of 27 more threads
2021-03-06 14:51:20,013 : INFO : worker thread finished; awaiting finish of 26 more threads
2021-03-06 14:51:20,169 : INFO : worker thread finished; awaiting finish of 25 more threads
2021-03-06 14:51:20,305 : INFO : worker thread finished; awaiting finish of 24 more threads
2021-03-06 14:51:20,330 : INFO : worker thread finished; awaiting finish of 23 more threads
2021-03-06 14:51:20,434 : INFO : worker thread finished; awaiting finish of 22 more threads
2021-03-06 14:51:20,761 : INFO : EPOCH 24 - PROGRESS: at 91.69% examples, 13976 words/s, in_qsize 21, out_qsize 1
2021-03-06 14:51:20,765 : INFO : wor

2021-03-06 14:53:29,427 : INFO : EPOCH 25 - PROGRESS: at 44.94% examples, 10655 words/s, in_qsize 80, out_qsize 2
2021-03-06 14:53:37,413 : INFO : EPOCH 25 - PROGRESS: at 45.61% examples, 10178 words/s, in_qsize 79, out_qsize 1
2021-03-06 14:53:47,818 : INFO : EPOCH 25 - PROGRESS: at 45.99% examples, 9542 words/s, in_qsize 80, out_qsize 0
2021-03-06 14:53:52,020 : INFO : EPOCH 25 - PROGRESS: at 46.23% examples, 9320 words/s, in_qsize 79, out_qsize 0
2021-03-06 14:53:53,382 : INFO : EPOCH 25 - PROGRESS: at 46.92% examples, 9370 words/s, in_qsize 80, out_qsize 0
2021-03-06 14:53:55,559 : INFO : EPOCH 25 - PROGRESS: at 47.38% examples, 9325 words/s, in_qsize 78, out_qsize 1
2021-03-06 14:53:57,010 : INFO : EPOCH 25 - PROGRESS: at 48.48% examples, 9463 words/s, in_qsize 80, out_qsize 0
2021-03-06 14:53:58,107 : INFO : EPOCH 25 - PROGRESS: at 49.67% examples, 9614 words/s, in_qsize 80, out_qsize 0
2021-03-06 14:54:01,281 : INFO : EPOCH 25 - PROGRESS: at 49.93% examples, 9467 words/s, in_qsi

2021-03-06 14:55:31,828 : INFO : worker thread finished; awaiting finish of 16 more threads
2021-03-06 14:55:31,853 : INFO : EPOCH 25 - PROGRESS: at 94.12% examples, 10573 words/s, in_qsize 15, out_qsize 1
2021-03-06 14:55:31,909 : INFO : worker thread finished; awaiting finish of 15 more threads
2021-03-06 14:55:31,977 : INFO : worker thread finished; awaiting finish of 14 more threads
2021-03-06 14:55:32,011 : INFO : worker thread finished; awaiting finish of 13 more threads
2021-03-06 14:55:32,184 : INFO : worker thread finished; awaiting finish of 12 more threads
2021-03-06 14:55:32,235 : INFO : worker thread finished; awaiting finish of 11 more threads
2021-03-06 14:55:32,347 : INFO : worker thread finished; awaiting finish of 10 more threads
2021-03-06 14:55:32,463 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-06 14:55:32,488 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-06 14:55:32,549 : INFO : worker thread finished; awa

Done.


#### 文書ベクトル作成用関数の定義
単語の和の平均で文書ベクトルを作成

In [4]:
def wordvec2docvec(sentence):
    # 文章ベクトルの初期値（0ベクトルを初期値とする）
    docvecs = np.zeros(num_features, dtype="float32")

    # 文章に現れる単語のうち、モデルに存在しない単語をカウントする
    denomenator = len(sentence)

    # 文章内の各単語ベクトルを足し合わせる
    for word in sentence:
        try:
            temp = model[word]
        except:
            denomenator -= 1
            continue
        docvecs += temp

    # 文章に現れる単語のうち、モデルに存在した単語の数で割る
    if denomenator > 0:
        docvecs =  docvecs / denomenator

    return docvecs

#### データ準備②

In [5]:
from sklearn.model_selection import train_test_split

print(len(datasets["document"]))
X, Y = [], []
for doc, category in tqdm(zip(datasets["document"], datasets["category"])):
    wakati = make_wakati(doc)
    docvec = wordvec2docvec(wakati)
    X.append(list(docvec))
    Y.append(category2id[category])
data_X = pd.DataFrame(X, columns=["X" + str(i + 1) for i in range(num_features)])
data_Y = pd.DataFrame(Y, columns=["category_id"])

train_x, test_x, train_y, test_y = train_test_split(data_X, data_Y, train_size= 0.7)

7376


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


#### XGBoostで分類器を作成＆予測

In [8]:
import xgboost as xgb
from sklearn.metrics import classification_report

print("Fitting XGboost model ...")
xgb_model = xgb.XGBClassifier()
xgb_model.fit(train_x, train_y)
print("Done.")

# 予測
pred = xgb_model.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))

Fitting XGboost model ...


/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:59:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Done.
                precision    recall  f1-score   support

dokujo-tsushin       0.89      0.83      0.86       277
  it-life-hack       0.93      0.91      0.92       252
 kaden-channel       0.90      0.95      0.92       247
livedoor-homme       0.71      0.87      0.78       125
   movie-enter       0.94      0.91      0.93       279
        peachy       0.81      0.84      0.82       247
          smax       0.96      0.95      0.95       248
  sports-watch       0.93      0.92      0.93       275
    topic-news       0.92      0.88      0.90       263

      accuracy                           0.90      2213
     macro avg       0.89      0.89      0.89      2213
  weighted avg       0.90      0.90      0.90      2213



#### LightGBMで分類器の作成＆予測

In [9]:
import lightgbm as lgbm

print("Fitting LightGBM model ...")
lgbm_model = lgbm.LGBMClassifier()
lgbm_model.fit(train_x, train_y)
print("Done.")

# 予測
pred = lgbm_model.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))

Fitting LightGBM model ...
Done.
                precision    recall  f1-score   support

dokujo-tsushin       0.88      0.83      0.85       274
  it-life-hack       0.91      0.93      0.92       243
 kaden-channel       0.92      0.94      0.93       253
livedoor-homme       0.69      0.86      0.77       123
   movie-enter       0.95      0.92      0.93       277
        peachy       0.81      0.80      0.80       264
          smax       0.96      0.96      0.96       247
  sports-watch       0.93      0.92      0.92       278
    topic-news       0.89      0.87      0.88       254

      accuracy                           0.89      2213
     macro avg       0.88      0.89      0.89      2213
  weighted avg       0.89      0.89      0.89      2213



#### Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(train_x, train_y)
pred = rfc.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


                precision    recall  f1-score   support

dokujo-tsushin       0.85      0.77      0.81       284
  it-life-hack       0.85      0.87      0.86       242
 kaden-channel       0.87      0.87      0.87       260
livedoor-homme       0.53      0.89      0.66        91
   movie-enter       0.93      0.87      0.90       289
        peachy       0.75      0.77      0.76       252
          smax       0.96      0.95      0.96       247
  sports-watch       0.91      0.90      0.90       277
    topic-news       0.88      0.81      0.84       271

      accuracy                           0.85      2213
     macro avg       0.84      0.85      0.84      2213
  weighted avg       0.86      0.85      0.85      2213



#### TF-IDFによる文書ベクトル

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = datasets["document"]
tfidf_vectorizer = TfidfVectorizer(analyzer=make_wakati)
tfidfs = tfidf_vectorizer.fit_transform(corpus)
print(tfidfs.shape)
# (7376, 79673)

tfidf_data_X = pd.DataFrame(tfidfs.toarray(),columns=["X"+str(i) for i in range(tfidfs.shape[1])])
train_x, test_x, train_y, test_y = train_test_split(data_X, data_Y, train_size=0.7)
lgbm_model = lgbm.LGBMClassifier()
lgbm_model.fit(train_x, train_y)
pred = lgbm_model.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))

(7376, 59937)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                precision    recall  f1-score   support

dokujo-tsushin       0.86      0.87      0.86       262
  it-life-hack       0.86      0.87      0.86       271
 kaden-channel       0.86      0.92      0.89       222
livedoor-homme       0.67      0.81      0.74       127
   movie-enter       0.94      0.88      0.91       271
        peachy       0.81      0.78      0.79       268
          smax       0.92      0.89      0.91       274
  sports-watch       0.94      0.93      0.94       270
    topic-news       0.91      0.89      0.90       248

      accuracy                           0.87      2213
     macro avg       0.86      0.87      0.87      2213
  weighted avg       0.88      0.87      0.87      2213

